# **Sentinel-2 SR Cloud/Shadow-free Project**

In [ ]:
# These codes are a modification of the Sentinel-2 Cloud Masking with s2cloudless
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
# Original source: https://github.com/google/earthengine-community/blob/master/tutorials/sentinel-2-s2cloudless/index.ipynb
# Credit: Justin Braaten (https://github.com/jdbcode)

In [ ]:
# Code modified by Syam'ani (https://github.com/syamaniulm)

In [ ]:
"""
These codes are intended to be executed in the JupyterLab environment installed on the user's computer,
not in the Google Coolab cloud environment. If these codes are to be executed in Google Colab,
then slight modifications have to be made to the codes.
"""

In [ ]:
"""
Python prerequisites: Python 3.9 or higher

Required packages:
earthengine-api (https://anaconda.org/conda-forge/earthengine-api)
geemap (https://anaconda.org/conda-forge/geemap)
geopandas (https://anaconda.org/conda-forge/geopandas)
json (https://anaconda.org/jmcmurray/json)
folium (https://anaconda.org/conda-forge/folium)
geedim (https://anaconda.org/conda-forge/geedim)

Other requirements:
You must have a Google Earth Engine account to run these codes.
"""

In [ ]:
import ee

In [ ]:
# Authenticate using your Google Earth Engine account
ee.Authenticate()

In [ ]:
# Initialize your Google Earth Engine project
# You must change 'your-ee-cloud-project' to match your Google Cloud project, e.g. 'ee-mygeeproject'
ee.Initialize(project='your-ee-cloud-project')

**Importing a region shapefile**

In [ ]:
# The shapefile must be projected using the Geographic Coordinate System (GCS)
# Replace the shapefile with your area of interest and adjust the path

import geopandas as gpd

region_shp = gpd.read_file('/Your_file_path/Your_shapefile_file_name.shp')
region_shp.plot()

In [ ]:
# Converting research area into Earth Engine Geometry

import json

region_js = json.loads(region_shp.to_json())
region_fc = ee.FeatureCollection(region_js)
region = ee.Geometry(region_fc.geometry())

**Creating cloud/shadow-free Sentinel-2 surface reflectance multitemporal mosaic**

In [ ]:
# Define some parameters
# You can change the following parameters as desired

AOI = ee.Geometry.centroid(region)
START_DATE = '2024-01-01'
END_DATE = '2024-06-30'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50
ZOOM_LEVEL = 12

In [ ]:
# Accessing the Sentinel-2 MSI TOC image collection

# Adjust the cloud percentage threshold as needed, e.g. 20%
cloud_ratio = 20

s2_col = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(START_DATE,END_DATE)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',cloud_ratio))
)

In [ ]:
# Sentinel-2 MSI RGB visualization

import geemap

rgb_vis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

map = geemap.Map()
map.centerObject(AOI,ZOOM_LEVEL)
map.add_layer(s2_col.median(), rgb_vis, 'Sentinel-2 RGB')
map

In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [ ]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(region, START_DATE, END_DATE)

In [ ]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [ ]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=ZOOM_LEVEL)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [ ]:
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

In [ ]:
display_cloud_layers(s2_sr_cld_col_eval_disp)

In [ ]:
s2_sr_cld_col = get_s2_sr_cld_col(region, START_DATE, END_DATE)

In [ ]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
s2_sr_median = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median()

In [ ]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=ZOOM_LEVEL)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

**Clipping image using region of interest**

In [ ]:
s2_image = s2_sr_median.clip(region)

In [ ]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=ZOOM_LEVEL)

# Add layers to the folium map.
m.add_ee_layer(s2_image,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'Region', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

**Create a fishnet to slice the large image into tiles**

In [ ]:
# Adjust the number of rows and the number of columns according to your region and needs
row_num = 3
col_num = 3

# Creating fishnet features
features = geemap.fishnet(region, rows=row_num, cols=col_num)

# Fishnet visualization
map = geemap.Map()
map.addLayer(features, {}, "Grids")
map.centerObject(region, ZOOM_LEVEL)
map

**Download image tiles into local drive**

In [ ]:
# Please be aware, the download process can take up to several hours.
# Depending on the size of the image and the number of bands you are downloading.
# And make sure your local drive capacity is sufficient to accommodate the tile image files.

In [ ]:
# Set the output directory, adjust it to your needs
out_dir = '/Your_image_tiles_output_folder'

# Select the required bands, adjust it to your needs
s2_image = s2_image.select('B2','B3','B4')

# Download image tiles into output directory
# Change prefix, crs, and scale according to your needs
geemap.download_ee_image_tiles(
    s2_image, features, out_dir, prefix='S2_SR_image_tile_', crs='EPSG:32750', scale=10
)